一.数据预处理

In [1]:
#加载图片数据
import os
from PIL import Image
imgs=[]
for i in range(1,5130):
    path='./data/'+str(i)+'.jpg'
    if os.path.exists(path):
        img=Image.open(path)   
        img=img.resize((227,227))
        imgs.append((i,img))

In [2]:
#将图片数据转化为tensor
import torch
import numpy as np
a=[]
for i,j in imgs:
    a.append(np.array(j))
b=torch.tensor(a)
imgs_vector=[]
for i in range(4869):
    imgs_vector.append((imgs[i][0],b[i]))

<ipython-input-2-4f612adcdbe8>:7: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\torch\csrc\utils\tensor_new.cpp:210.)
  b=torch.tensor(a)


In [3]:
#加载文本数据
sentences=[]
for i in range(1,5130):
    path='./data/'+str(i)+'.txt'
    if os.path.exists(path):
        with open(path,'r',errors='ignore') as f:
            sentence=f.read()
            sentences.append((i,sentence))

In [6]:
#组合图片数据和文本数据
imgs_vector_=[]
for i in range(4869):
    imgs_vector_.append((imgs_vector[i][0],imgs_vector[i][1].transpose(0,2)*1.0))
data=[]
for i in range(4869):
    data.append((imgs_vector[i][0],imgs_vector_[i][1],sentences[i][1]))

In [7]:
#划分训练集和测试集
train=[]
with open('train.txt','r') as f:
    for line in f.readlines():
        line=line.strip('\n')  
        train.append(line)
train_=[]
for i in train:
    if i!='guid,tag':
        a=i.index(',')
        train_.append((int(i[:a]),i[a+1:]))
train__=[]
for i in train_:
    if i[1]=='negative':
        train__.append((i[0],0))
    if i[1]=='neutral':
        train__.append((i[0],1))
    if i[1]=='positive':
        train__.append((i[0],2))
test=[]
with open('test_without_label.txt','r') as f:
    for line in f.readlines():
        line=line.strip('\n')  
        test.append(line)
test_=[]
for i in test:
    if i!='guid,tag':
        a=i.index(',')
        test_.append((int(i[:a]),-1))
dict_train={}
for i in train__:
    dict_train[i[0]]=i[1]
dict_test={}
for i in test_:
    dict_test[i[0]]=-1
train_data=[]
test_data=[]
for i in data:
    if i[0] in dict_train:
        train_data.append((i[0],i[1],i[2],dict_train[i[0]]))
    if i[0] in dict_test:
        test_data.append((i[0],i[1],i[2],dict_test[i[0]]))
    else: 
        continue      

In [8]:
#将文本数据转化为tensor
import nltk
from nltk.corpus import stopwords
from gensim.models import Word2Vec
train_sentences=[]
for i in train_data:
    train_sentences.append(i[2])
train_sentences_fenci=[]
for train_sentence in train_sentences:
    train_sentences_fenci.append(nltk.word_tokenize(train_sentence))
train_sentences_quci=[]
stop_words=stopwords.words('english')
for i in [',','.',';','?','!','#','&','(',')','@',':','-','...','``','....','[',']','http','“','--','/…',"''","'",'..','$','%','…','–','‐‐‐','=','<','>','[]','http…','’','~','RT']:
    stop_words.append(i)
for train_sentence in train_sentences_fenci:
    quci=[i for i in train_sentence if i not in stop_words and i[:2]!='//']
    train_sentences_quci.append(quci)  
train_sentences_small=[]
for train_sentence in train_sentences_quci:
    train_sentences_small_=[]
    for i in train_sentence:
         train_sentences_small_.append(i.lower())
    train_sentences_small.append(train_sentences_small_)
model1=Word2Vec(train_sentences_small,sg=1,window=5,min_count=5,negative=3,sample=0.001,hs=1,workers=4)
words=list(model1.wv.index_to_key)
train_set=[]
for i in range(4000):
    if len(train_sentences_small[i])==0:
        train_set.append((train_data[i][0],train_data[i][1],np.zeros(100),train_data[i][3]))
    else:
        a=np.zeros(100)
        for j in train_sentences_small[i]:
            if j in words:
                a+=model1.wv[j]
        a=a/len(train_sentences_small[i])
        train_set.append((train_data[i][0],train_data[i][1],a,train_data[i][3]))

In [9]:
#从训练集中划分训练集和验证集
train_sets=train_set[:3000]
dev_sets=train_set[3000:]
train_sets=torch.utils.data.DataLoader(train_sets,batch_size=64,shuffle=True)      
dev_sets=torch.utils.data.DataLoader(dev_sets,batch_size=64,shuffle=True)       

二.搭建网络

In [10]:
from torch import nn
class Model(nn.Module):
    def __init__(self):
        super(Model,self).__init__()
        self.conv1=nn.Sequential(                                                                       
        nn.Conv2d(in_channels=3,out_channels=96,kernel_size=11,stride=4,padding=0,bias=True),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=3,stride=2),
        nn.LocalResponseNorm(size=5,alpha=0.0001,beta=0.75)
                                 )
        self.conv2=nn.Sequential(                                                                       
        nn.Conv2d(in_channels=96,out_channels=256,kernel_size=5,stride=1,padding=2,bias=True),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=3,stride=2),
        nn.LocalResponseNorm(size=5,alpha=0.0001,beta=0.75) 
                                 )
        self.conv3=nn.Sequential(                                                                        
        nn.Conv2d(in_channels=256,out_channels=384,kernel_size=3,stride=1,padding=1,bias=True),
        nn.ReLU()       
                                 )        
        self.conv4=nn.Sequential(                                                                                 
        nn.Conv2d(in_channels=384,out_channels=384,kernel_size=3,stride=1,padding=1,bias=True),
        nn.ReLU()                  
                                 )              
        self.conv5=nn.Sequential(                                                                        
        nn.Conv2d(in_channels=384,out_channels=256,kernel_size=3,stride=1,padding=1,bias=True),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=3,stride=2)      
                                 )                                                                                                                 #网络第六层                           
        self.flatten=nn.Flatten()
        self.f1=nn.Sequential(                                                                             
        nn.Linear(9216,4096),                                                            
        nn.ReLU(),
        nn.Dropout()
                              )
        self.f2=nn.Sequential(                                                                           
        nn.Linear(4096,4096),                                                            
        nn.ReLU(),
        nn.Dropout()      
                              )
        self.f3=nn.Linear(4096,64)      
        self.f4=nn.Sequential(                                                                           
        nn.Linear(100,100),                                                            
        nn.ReLU(),
        nn.Dropout(),  
        nn.Linear(100,64)
                              )    
        self.f5=nn.Sequential(                                                                           
        nn.Linear(128,32),                                                            
        nn.ReLU(),
        nn.Dropout(),  
        nn.Linear(32,3)
                              )        
    def forward(self,img,sentence):                                                                                                       
        img=self.conv1(img)
        img=self.conv2(img)
        img=self.conv3(img)
        img=self.conv4(img)
        img=self.conv5(img)
        img=self.flatten(img)
        img=self.f1(img)
        img=self.f2(img)
        img=self.f3(img)
        sentence=sentence.to(torch.float32)
        sentence=self.f4(sentence)
        feature=torch.cat([img,sentence],1)
        pred=self.f5(feature)
        return pred

三.训练网络

In [11]:
epoch=5                                                                                     
model2=Model()                                                                                      
optimizer=torch.optim.SGD(model2.parameters(),lr=0.01)                                                
loss_f=nn.CrossEntropyLoss()                                                                       
for epoch in range(1,epoch+1):                                   
    for train_index,train_img,train_sentence,train_label in train_sets:
        y_hat=model2(train_img,train_sentence) 
        loss=loss_f(y_hat,train_label)
        optimizer.zero_grad()                                                                        
        loss.backward()                                                                              
        optimizer.step()                                                                            
    dev_loss=0                                                                                      
    dev_acc=0                                                                                    
    for dev_index,dev_img,dev_sentence,dev_label in dev_sets  : 
        y_hat=model2(dev_img,dev_sentence)                                                                         
        dev_loss+=loss_f(y_hat,dev_label).item()*dev_index.size(0)                                                   
        dev_acc+=(y_hat.argmax(dim=1)==dev_label).float().sum().item()                 
    print('epoch:%d dev_loss:%.6f dev_acc:%.4f'%(epoch,dev_loss/1000,dev_acc/1000))                   

epoch:1 dev_loss:0.937666 dev_acc:0.5500


四.预测

In [12]:
test_sentences=[]
for i in test_data:
    test_sentences.append(i[2])
test_sentences_fenci=[]
for test_sentence in test_sentences:
    test_sentences_fenci.append(nltk.word_tokenize(test_sentence))
test_sentences_quci=[]
for test_sentence in test_sentences_fenci:
    quci=[i for i in test_sentence if i not in stop_words and i[:2]!='//']
    test_sentences_quci.append(quci)  
test_sentences_small=[]
for test_sentence in test_sentences_quci:
    test_sentences_small_=[]
    for i in test_sentence:
         test_sentences_small_.append(i.lower())
    test_sentences_small.append(test_sentences_small_)
test_set=[]
for i in range(511):
    if len(test_sentences_small[i])==0:
        test_set.append((test_data[i][0],test_data[i][1],np.zeros(100),test_data[i][3]))
    else:
        a=np.zeros(100)
        for j in test_sentences_small[i]:
            if j in words:
                a+=model1.wv[j]
        a=a/len(test_sentences_small[i])
        test_set.append((test_data[i][0],test_data[i][1],a,test_data[i][3]))
test_set=torch.utils.data.DataLoader(test_set,batch_size=64,shuffle=True)
pred=[]
for test_index,test_img,test_sentence,test_label in test_set: 
        y_hat=model2(test_img,test_sentence)
        for i in range(len(test_index)):
            pred.append((test_index[i],y_hat.argmax(dim=1)[i]))

五.写入预测结果

In [25]:
a=[]
for i in test_:
    a.append(i[0])
b={}
for i in pred:
    b[int(i[0])]=int(i[1])
c=['guid,tag']
for i in a:
    if b[i]==0:
        c.append(str(i)+','+'negative')
    if b[i]==1:
        c.append(str(i)+','+'neutral')
    if b[i]==2:
        c.append(str(i)+','+'positive')
with open('pred.txt','w') as f:
    for i in c:
        f.write(i+'\n')
    f.close()